In [16]:
import pandas as pd

In [15]:
veri_seti = pd.read_csv('../datasets/train.csv')

C:\Users\Osman\AppData\Local\Temp\ipykernel_19088\3530611747.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  veri_seti = pd.read_csv('datasets/train.csv')


In [18]:
from sklearn.preprocessing import LabelEncoder


def bosverileri_doldur_train(veriseti):
  for kolon in veriseti.columns:
    if veriseti[kolon].isnull().any():
        en_cok_olan_deger = veriseti[kolon].value_counts().idxmax()
        veriseti[kolon].fillna(en_cok_olan_deger, inplace=True)
  return veriseti

veri_seti= bosverileri_doldur_train(veri_seti)
print(veri_seti.isnull().sum().sum())

def sayisal_veriye_donustur(veri_seti):

    y = veri_seti['LABEL']
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    for kolon in veri_seti.columns:
        if veri_seti[kolon].dtype == 'object':
            veri_seti[kolon] = veri_seti[kolon].astype(str)  # Veriyi string olarak dönüştür
            veri_seti[kolon] = label_encoder.fit_transform(veri_seti[kolon])


    X = veri_seti.drop(columns=['LABEL','MUSTERI_ID'])  # LABEL sütununu hariç tut

    return X , y

X , y= sayisal_veriye_donustur(veri_seti)


0


In [ ]:
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn.model_selection import train_test_split

# RFE için XGBoost sınıflandırıcı kullanalım
estimator = xgb.XGBClassifier()

# Özellik seçimi için RFE kullanarak bir model oluşturma
selector = RFE(estimator, n_features_to_select=5, step=1)

# en önemli özellikleri seçelme
selector = selector.fit(X, y)

# Seçilen özelliklerin indekslerini alama
selected_features_indexes = selector.get_support(indices=True)

# Seçilen özellikleri ve önem sıralarını yazdırma
print("Seçilen Özellikler ve Önem Sıraları:")
for i, index in enumerate(selected_features_indexes):
    feature = X.columns[index]
    importance_rank = selector.ranking_[index]
    print(f"{i+1}. {feature}: {importance_rank}")

# Seçilen özellikleri ve önem sıralarını bir DataFrame'e kaydetme
selected_features = X.columns[selected_features_indexes]
selected_features_df = pd.DataFrame({'Feature': selected_features, 'Information Gain': selector.ranking_[selected_features_indexes]})

# Özellikleri CSV dosyasına kaydedelim
selected_features_df.to_csv('feature_info_gain_df.csv', index=False)
